<a href="https://colab.research.google.com/github/NustinJewman/PythonFFB/blob/main/data_munging_notes_pt_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/fantasydatapros/data/74b84c5fb2371b954b52b4f67ae5220930d57861/fantasypros/fp_projections.csv')

In [ ]:
rb_df = df.loc[df['Pos'] == 'RB']

rb_df.sort_values(by='ReceivingYds', ascending=False).head(10)

In [ ]:
import seaborn as sns


In [ ]:
adp_df = pd.read_csv('https://raw.githubusercontent.com/fantasydatapros/data/master/fantasypros/adp/PPR_ADP.csv', index_col=0)


In [ ]:
adp_df.head(10)

In [59]:
adp_df['ADP_RANK'] = adp_df['AVG'].rank()
adp_df.head(5)

,PLAYER,POS,AVG,ADP_RANK
0,Christian McCaffrey,RB,1.0,1.0
1,Saquon Barkley,RB,2.0,2.0
2,Ezekiel Elliott,RB,3.5,3.0
3,Michael Thomas,WR,4.0,4.0
4,Dalvin Cook,RB,5.0,5.0


In [72]:
adp_df_cutoff = adp_df[:100] #limited to 100 rows. not the bums like trent taylor
adp_df_cutoff.shape

(100, 4)

In [73]:
replacement_players = {
    'RB': '',
    'WR': '',
    'QB': '',
    'TE': '',
}

for _, row in adp_df_cutoff.iterrows():
  position = row['POS']
  player = row['PLAYER']

  if position in replacement_players:
    replacement_players[position] = player

In [74]:
replacement_players

{'QB': 'Aaron Rodgers',
 'RB': 'Latavius Murray',
 'TE': 'Rob Gronkowski',
 'WR': 'Jamison Crowder'}

In [63]:
df = df[['Player', 'Pos', 'Team', 'FantasyPoints']]  #this is how to filter out columns in pandas. set new df original to df and columns you wish to keep in new df.
df.head()

,Player,Pos,Team,FantasyPoints
0,Lamar Jackson,QB,BAL,361.1
1,Patrick Mahomes,QB,KC,345.1
2,Dak Prescott,QB,DAL,318.5
3,Deshaun Watson,QB,HOU,312.7
4,Russell Wilson,QB,SEA,308.5


In [87]:
replacement_values = {}

for position, player_name in replacement_players.items():
  player = df.loc[df['Player'] == player_name]
  replacement_values[position] = player['FantasyPoints'].tolist()[0]

In [88]:
replacement_values

{'QB': 277.3, 'RB': 97.4, 'TE': 92.3, 'WR': 118.9}

In [ ]:

scoring_weights = {
    'receptions': 1, # PPR
    'receiving_yds': 0.1,
    'receiving_td': 6,
    'FL': -2, #fumbles lost
    'rushing_yds': 0.1,
    'rushing_td': 6,
    'passing_yds': 0.04,
    'passing_td': 4,
    'int': -2
}
df['FantasyPoints'] = (
    df['Receptions']*scoring_weights['receptions'] + df['ReceivingYds']*scoring_weights['receiving_yds'] + \
    df['ReceivingTD']*scoring_weights['receiving_td'] + df['FL']*scoring_weights['FL'] + \
    df['RushingYds']*scoring_weights['rushing_yds'] + df['RushingTD']*scoring_weights['rushing_td'] + \
    df['PassingYds']*scoring_weights['passing_yds'] + df['PassingTD']*scoring_weights['passing_td'] + \
    df['Int']*scoring_weights['int'] )

In [94]:
pd.set_option('chained_assignment', None)

df = df.loc[df['Pos'].isin(['QB', 'RB', 'WR', 'TE'])]

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html
df['VOR'] = df.apply(
    lambda row: row['FantasyPoints'] - replacement_values.get(row['Pos']), axis=1)

In [95]:
df.head()

,Player,Pos,Team,FantasyPoints,VOR
0,Lamar Jackson,QB,BAL,361.1,83.8
1,Patrick Mahomes,QB,KC,345.1,67.8
2,Dak Prescott,QB,DAL,318.5,41.2
3,Deshaun Watson,QB,HOU,312.7,35.4
4,Russell Wilson,QB,SEA,308.5,31.2


In [98]:
pd.set_option('display.max_rows', None)
df.sort_values(by='VOR',ascending=False).head(100)

,Player,Pos,Team,FantasyPoints,VOR
12,Christian McCaffrey,RB,CAR,272.2,174.8
23,Saquon Barkley,RB,NYG,244.6,147.2
25,Ezekiel Elliott,RB,DAL,240.3,142.9
26,Derrick Henry,RB,TEN,239.9,142.5
30,Dalvin Cook,RB,MIN,231.9,134.5
33,Alvin Kamara,RB,NO,213.3,115.9
35,Clyde Edwards-Helaire,RB,KC,203.1,105.7
36,Joe Mixon,RB,CIN,201.6,104.2
37,Nick Chubb,RB,CLE,200.0,102.6
38,Miles Sanders,RB,PHI,197.5,100.1
